<a href="https://colab.research.google.com/github/anakayam45/dataset_gunting_batu_kertas/blob/main/Klasifikasi_Gambar_Gunting_Batu_Kertas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten
import os, matplotlib.pyplot as plt, seaborn as sns, numpy as np, cv2, pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
!git clone "https://github.com/anakayam45/dataset_gunting_batu_kertas.git"
path_to_repo = '/content/dataset_gunting_batu_kertas'

In [ ]:
path_to_dataset = os.path.join(path_to_repo, "result")
df = tf.keras.utils.image_dataset_from_directory(path_to_dataset, image_size=(128, 128), batch_size=32, shuffle=True)

In [ ]:
for images, labels in df.take(1):
  plt.figure(figsize=(10, 10))
  for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(int(labels[i]))
    plt.axis("off")
plt.show()

In [ ]:
import cv2
import albumentations as A
!apt install albumentations
from albumentations import RandomBrightnessContrast, GaussianBlur, HorizontalFlip
from albumentations.core.composition import Compose, OneOf


for folder in (os.listdir(path_to_dataset)):
  folder_path = os.path.join(path_to_dataset, folder)
  for file in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file)
    if not os.path.isdir(file_path):

      flip = A.Compose([A.HorizontalFlip(p=1)])
      light = A.Compose([RandomBrightnessContrast(limit=(0.2, 0), p=1)])
      dark = A.Compose([RandomBrightnessContrast(limit=(-0.2, 0), p=1)])
      blurr = A.Compose([GaussianBlur(p=1)])

      if os.path.isfile(file_path):
        image = cv2.imread(file_path)

        augmented = flip(image=image)
        augmented_image = augmented['image']
        cv2.imwrite(os.path.join(folder_path, f"{len(os.listdir(folder_path))+1}_flip.jpg"), augmented_image)

        augmented = light(image=image)
        augmented_image = augmented['image']
        cv2.imwrite(os.path.join(folder_path, f"{len(os.listdir(folder_path))+1}_light.jpg"), augmented_image)

        augmented = dark(image=image)
        augmented_image = augmented['image']
        cv2.imwrite(os.path.join(folder_path, f"{len(os.listdir(folder_path))+1}_dark.jpg"), augmented_image)

        augmented = blurr(image=image)
        augmented_image = augmented['image']
        cv2.imwrite(os.path.join(folder_path, f"{len(os.listdir(folder_path))+1}_blurr.jpg"), augmented_image)


In [ ]:
path_to_dataset = os.path.join(path_to_repo, "result")
df = tf.keras.utils.image_dataset_from_directory(path_to_dataset, image_size=(128, 128), batch_size=32, shuffle=True)
len(os.listdir(os.path.join(path_to_dataset, "gunting")))

In [ ]:
for images, labels in df.take(2):
  plt.figure(figsize=(10, 10))
  for i in range(3):
    plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(int(labels[i]))
    plt.axis("off")
plt.show()

In [8]:
# Pembagian data
train_size = 0.8
val_size = 0.1
test_size = 0.1

# Hitung jumlah batch
total_batches = len(df)
train_batches = int(total_batches * train_size)
val_batches = int(total_batches * val_size)

# Pembagian dataset
train_ds = df.take(train_batches)
val_ds = df.skip(train_batches).take(val_batches)
test_ds = df.skip(train_batches + val_batches)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(df.class_names), activation='softmax')  # Sesuaikan jumlah kategori
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_ds, epochs=10, validation_data=val_ds)

In [ ]:
!pip install gradio
import gradio as gr

In [ ]:
def process_image(image):
    if image is None:
        return "No image uploaded"
    image = image.resize((128, 128))
    image = np.array(image)
    image = np.expand_dims(image, axis=0)
    predict = model.predict(image)
    predicted_class_index = np.argmax(predict)
    predicted_class_name = df.class_names[predicted_class_index]
    return predicted_class_name

demo = gr.Interface(
    fn = process_image,
    inputs = gr.Image(type="pil", height=128, width=128),
    outputs = "text",
    title="Gunting Batu Kertas",
    description="Upload gambar tangan anda membentuk gunting batu kertas"
)

demo.launch(debug=True)

In [ ]:
model.summary()